<a href="https://colab.research.google.com/github/v-demydchenko/Python-for-DA/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22HW_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90_%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%96%D0%B2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cookie_cats.csv')

In [5]:
import math
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

p1 = 0.19
p2 = 0.20
alpha = 0.05
power = 0.8

effect_size = proportion_effectsize(p1, p2)

analysis = NormalIndPower()

sample_size_per_group = analysis.solve_power(effect_size=effect_size,
                                             alpha=alpha,
                                             power=power,
                                             alternative='two-sided')

total_sample_size = math.ceil(sample_size_per_group * 2)

print(f"Необхідно користувачів у кожній групі: {math.ceil(sample_size_per_group)}")
print(f"Загалом користувачів: {total_sample_size}")

Необхідно користувачів у кожній групі: 24638
Загалом користувачів: 49276


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [11]:
retention_7_means = df.groupby('version')['retention_7'].mean()

print("Середнє значення retention_7 по версіях:")
print(retention_7_means)
print("\nРізниця між версіями:")
print(f"gate_30: {retention_7_means['gate_30']:.4f}")
print(f"gate_40: {retention_7_means['gate_40']:.4f}")
print(f"Різниця: {retention_7_means['gate_30'] - retention_7_means['gate_40']:.4f}")

# Версія gate_30 дає краще утримання через 7 днів після встановлення гри, так як
# має вищий показник утримання на 7-й день - 19.02%
# Версія з воротами на 40-му рівні (gate_40) має нижчий показник - 18.20%
# Різниця становить 0.82 процентних пункти на користь версії gate_30

Середнє значення retention_7 по версіях:
version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

Різниця між версіями:
gate_30: 0.1902
gate_40: 0.1820
Різниця: 0.0082


3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [9]:
retention_means = df.groupby("version")["retention_7"].mean()
retention_means

# 1. Чи є статистично значуща різниця між поведінкою користувачів у різних версіях гри?
# Так, оскільки p-value < 0.05. Різниця між групами значна.
#  2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?
# Ні, інтервали не перетинаються. Це підтверджує, що утримання у групах відрізняється.

,retention_7
version,
gate_30,0.190201
gate_40,0.182000


4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.
